## Output Parser

1. 引入库

In [1]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pprint import pprint
from chat_model_client import get_model

2. 声明返回结构

In [2]:
class Marathon(BaseModel):
    name: str = Field(
        description="name of event"
    ),
    date: str = Field(
        description="the date of event"
    ),
    city: str = Field(
        description="Event host city"
    )

3.解析器及prompt

In [3]:
query = "随机生成一个马拉松赛事，并给出比赛时间及城市"
parser = PydanticOutputParser(pydantic_object=Marathon)
prompt = PromptTemplate(
    template="回答下面的问题：\n{format_instructions}\n{query}\n如果输出的是代码块，请不要包括首尾的```符号",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='name of event'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the date of event'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


4. Prompt

In [7]:
pprint(prompt.format(query=query))

('回答下面的问题：\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"name": {"title": "Name", "type": "string"}, "date": '
 '{"title": "Date", "type": "string"}, "city": {"description": "Event host '
 'city", "title": "City", "type": "string"}}, "required": ["city"]}\n'
 '```\n'
 '随机生成一个马拉松赛事，并给出比赛时间及城市\n'
 '如果输出的是代码块，请不要包括首尾的```符号')


5. 解析并输出内容

In [4]:
llm = get_model("openai")

chain = prompt | llm | parser
output= chain.invoke(query)

pprint(output)

/var/folders/5x/c0q41fpx6l540lsl42_bzk5h0000gq/T/ipykernel_68319/1734759378.py:1: UserWarning: Parameters {'presence_penalty', 'top_p', 'frequency_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  llm = get_model("openai")


Marathon(name='New York City Marathon', date='November 7, 2021', city='New York City')
